In [29]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [30]:
df = pd.read_csv('test.csv')
dft = pd.read_csv('todascenario2.csv')

In [31]:
df

,full_name,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,mileage,body_type,city,ID
0,2017 Hyundai Grand i10 1.2 Kappa Sportz Option,Jun 2017,1197 cc,Comprehensive,Manual,"53,658 Kms",First Owner,Petrol,81.86bhp,5.0,18.9 kmpl,Hatchback,Mumbai,0
1,2020 Maruti Swift AMT VXI,Feb 2021,1197 cc,Not Available,Automatic,"33,055 Kms",Second Owner,Petrol,81.80bhp,5.0,21.21 kmpl,Hatchback,Jaipur,1
2,2010 Maruti Zen Estilo LX BSIII,Jul 2010,998 cc,Third Party,Manual,"90,385 Kms",Third Owner,Petrol,67.1bhp,5.0,18.2 kmpl,Hatchback,Chandigarh,2
3,2010 Maruti Alto LXi,2010,796 cc,Third Party insurance,Manual,"1,10,000 Kms",First Owner,Petrol,46.3bhp,5.0,19.7 kmpl,Hatchback,Delhi,3
4,2018 Hyundai i20 Active S Petrol,Oct 2018,1197 cc,Comprehensive,Manual,"58,000 Kms",Second Owner,Petrol,81.86bhp,5.0,17.19 kmpl,Hatchback,Delhi,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,2022 Tata Tigor EV XZ Plus,2022,NaN,Third Party insurance,Automatic,"30,000 Kms",First Owner,Electric,73.75bhp,5.0,NaN,Sedan,Pune,3485
3486,2017 Renault KWID 1.0 RXT,Apr 2017,999 cc,Third Party insurance,Manual,"64,024 Kms",First Owner,Petrol,67bhp,5.0,23.01 kmpl,Hatchback,Bangalore,3486
3487,2017 Honda City i VTEC VX,Dec 2017,1497 cc,Comprehensive,Manual,"39,000 Kms",First Owner,Petrol,117.3bhp,5.0,17.8 kmpl,Sedan,Delhi,3487
3488,2016 Honda Jazz 1.2 V i VTEC,Jun 2016,1199 cc,Comprehensive,Manual,"33,000 Kms",First Owner,Petrol,88.7bhp,5.0,18.7 kmpl,Hatchback,Mumbai,3488


In [32]:
import pandas as pd

# Carregar os DataFrames a partir dos arquivos CSV
df = pd.read_csv('test.csv')
dft = pd.read_csv('todascenario2MenosGrupos.csv')

# Ver as colunas de cada DataFrame
print("Colunas do DataFrame 'df':")
print(df.columns)

print("\nColunas do DataFrame 'dft':")
print(dft.columns)


Colunas do DataFrame 'df':
Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city', 'ID'],
      dtype='object')

Colunas do DataFrame 'dft':
Index(['id_transmissao', 'transmission_type', 'body_type', 'id_body_type',
       'modelo', 'id_modelos', 'marca', 'id_categoria_marca', 'ano', 'city',
       'id_city', 'insurance', 'id_insurance', 'seats', 'engine_capacity',
       'engine_capacity_group', 'id_engine_capacity_group', 'fuel_type',
       'id_fuel_type', 'kms_driven', 'kms_group', 'id_kms_group', 'max_power',
       'max_power_group', 'id_max_power_group', 'owner_type', 'id_owner_type',
       'resale_price_Lakh'],
      dtype='object')


In [33]:
# KMS __________________________________________________
#remove kms from kms_driven
df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
# Remove , and blanks from 'kms_driven'
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()
df = df.astype({
   
    'kms_driven': float,
   
})

In [34]:
# groups kms

limites = [0, 150000, 600000, float('inf')]  
rotulos = ['Até 150.000 km', '150.000 km a 600.000 km', 'Acima de 600.000 km']

df['kms_group'] = pd.cut(df['kms_driven'], bins=limites, labels=rotulos, right=False)

df.loc[df['kms_driven'] == 1200000, 'kms_group'] = 'Acima de 600.000 km'

In [35]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['kms_group'], dft['id_kms_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_kms_group'] = df['kms_group'].map(mapeamento_max_power_group)

In [36]:
# SEATS __________________________________________________
# Check for model cars with missing values in the 'seats' column
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

# Update number of seats based on conditions
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI']), 'seats'] = 5
df.loc[df['full_name'] == '2008 Ford Endeavour 3.0 4x4 Thunder Plus', 'seats'] = 7

Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2009 Maruti Estilo LXI
2012 Honda Jazz Select Edition


In [37]:
# Substituir os valores nulos na coluna 'seats' por 5
df['seats'].fillna(5, inplace=True)


In [38]:
# Substituir os valores 'Third Party insurance' por 'Third Party' em df
df['insurance'] = df['insurance'].replace('Third Party insurance', 'Third Party', regex=True)

# Substituir os valores NaN por 'No Insurance' em df
df['insurance'].fillna('No Insurance', inplace=True)

# Visualizar a coluna 'insurance' atualizada em df
print(df['insurance'].unique())
print(dft['id_insurance'].unique())

['Comprehensive' 'Not Available' 'Third Party' 'Zero Dep' 'No Insurance']
[0.25663332 1.30640077 0.44540821 1.41864433 3.34976304]


In [39]:

print(df['insurance'].unique())
print(dft['insurance'].unique())

['Comprehensive' 'Not Available' 'Third Party' 'Zero Dep' 'No Insurance']
['Third Party' 'Zero Dep' 'Comprehensive' 'Not Available' 'No Insurance']


In [40]:
# Mapear os valores desejados em 'df' para os valores correspondentes em 'dft'
mapeamento_valores = {
    'Third Party insurance': 'Third Party',
    pd.NA: 'No Insurance'
}

# Aplicar as transformações nos valores da coluna 'insurance' em 'df'
df['insurance'] = df['insurance'].replace(mapeamento_valores).fillna('No Insurance')

# Visualizar os valores únicos após as transformações em 'df'
print(df['insurance'].unique())

['Comprehensive' 'Not Available' 'Third Party' 'Zero Dep' 'No Insurance']


In [41]:
# ENGINE __________________________________________________
#remove cc from engine_capacity
df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')

df = df.astype({
    
    'engine_capacity': float,
  
})

# groups engine_capacity

limites = [0,1, 1000, 3000, float('inf')]  
rotulos = ['Elétrico', 'até 1000 cc', 'até 3000 cc', 'mais de 3000 cc']

df['engine_capacity_group'] = pd.cut(df['engine_capacity'], bins=limites, labels=rotulos, right=False)



In [42]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['engine_capacity_group'], dft['id_engine_capacity_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_engine_capacity_group'] = df['engine_capacity_group'].map(mapeamento_max_power_group)

In [43]:
# MAX POWER 
# #convert to bhp__________________________________________________
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)


# Convert not NaN to float
Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]

df['max_power'] = Correct_Max_Power

#Create grupos
#Create grupos

# groups max_power


limites = [0, 200, 400, 600]  
rotulos = ['0-200 bhp', '200-400 bhp', '400-600 bhp']

df['max_power_group'] = pd.cut(df['max_power'], bins=limites, labels=rotulos, right=False)

In [44]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['max_power_group'], dft['id_max_power_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_max_power_group'] = df['max_power_group'].map(mapeamento_max_power_group)


In [45]:
df

,full_name,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,mileage,body_type,city,ID,kms_group,id_kms_group,engine_capacity_group,id_engine_capacity_group,max_power_group,id_max_power_group
0,2017 Hyundai Grand i10 1.2 Kappa Sportz Option,Jun 2017,1197.0,Comprehensive,Manual,53658.0,First Owner,Petrol,81.8,5.0,18.9 kmpl,Hatchback,Mumbai,0,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055
1,2020 Maruti Swift AMT VXI,Feb 2021,1197.0,Not Available,Automatic,33055.0,Second Owner,Petrol,81.8,5.0,21.21 kmpl,Hatchback,Jaipur,1,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055
2,2010 Maruti Zen Estilo LX BSIII,Jul 2010,998.0,Third Party,Manual,90385.0,Third Owner,Petrol,67.1,5.0,18.2 kmpl,Hatchback,Chandigarh,2,Até 150.000 km,0.005469,até 1000 cc,0.776698,0-200 bhp,0.009055
3,2010 Maruti Alto LXi,2010,796.0,Third Party,Manual,110000.0,First Owner,Petrol,46.3,5.0,19.7 kmpl,Hatchback,Delhi,3,Até 150.000 km,0.005469,até 1000 cc,0.776698,0-200 bhp,0.009055
4,2018 Hyundai i20 Active S Petrol,Oct 2018,1197.0,Comprehensive,Manual,58000.0,Second Owner,Petrol,81.8,5.0,17.19 kmpl,Hatchback,Delhi,4,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,2022 Tata Tigor EV XZ Plus,2022,NaN,Third Party,Automatic,30000.0,First Owner,Electric,73.7,5.0,NaN,Sedan,Pune,3485,Até 150.000 km,0.005469,NaN,4.127914,0-200 bhp,0.009055
3486,2017 Renault KWID 1.0 RXT,Apr 2017,999.0,Third Party,Manual,64024.0,First Owner,Petrol,67.0,5.0,23.01 kmpl,Hatchback,Bangalore,3486,Até 150.000 km,0.005469,até 1000 cc,0.776698,0-200 bhp,0.009055
3487,2017 Honda City i VTEC VX,Dec 2017,1497.0,Comprehensive,Manual,39000.0,First Owner,Petrol,117.3,5.0,17.8 kmpl,Sedan,Delhi,3487,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055
3488,2016 Honda Jazz 1.2 V i VTEC,Jun 2016,1199.0,Comprehensive,Manual,33000.0,First Owner,Petrol,88.7,5.0,18.7 kmpl,Hatchback,Mumbai,3488,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055


In [46]:
df['max_power_group'] = df['max_power_group'].astype('string')
df['engine_capacity_group'] = df['engine_capacity_group'].astype('string')
print(df.dtypes)


full_name                           object
registered_year                     object
engine_capacity                    float64
insurance                           object
transmission_type                   object
kms_driven                         float64
owner_type                          object
fuel_type                           object
max_power                          float64
seats                              float64
mileage                             object
body_type                           object
city                                object
ID                                   int64
kms_group                         category
id_kms_group                      category
engine_capacity_group       string[python]
id_engine_capacity_group           float64
max_power_group             string[python]
id_max_power_group                category
dtype: object


In [47]:
#BODY TYPE ----------------------
#Replace the missing types and the ones that were wrong
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI', '2019 Maruti Wagon R ZXI AMT 1.2BSIV',
                             '2016 Mercedes-Benz B Class B180 Sport', '2016 Mercedes-Benz A Class A200 D Sport Edition',
                             '2004 Tata Indigo Marina GLX', '2017 Volvo V40 D3 R Design']), 'body_type'] = 'Hatchback'
df.loc[df['full_name'].isin(['2005 Tata Sumo Victa EX 10/7 Str BSII', '2008 Tata Sumo CX', '2010 Tata New Safari DICOR 2.2 EX 4x2', 
                             '2017 Mahindra KUV 100 mFALCON G80 K8', '2019 Mahindra Bolero Camper GOLD ZX 2WD']), 'body_type'] = 'SUV'
df.loc[df['full_name'].isin(['2015 Honda Mobilio E i-DTEC', '2018 Maruti Eeco 5 Seater AC BSIV', '2016 Datsun GO Plus A']), 'body_type'] = 'MUV'
df.loc[df['full_name'].isin(['2021 BMW 7 Series 730Ld DPE Signature','2006 Hyundai Accent CRDi', '2007 Maruti SX4 Vxi BSIII',
                             '2020 Mercedes-Benz S-Class Maybach S560', '2004 Skoda Octavia 1.8 Turbo Petrol RS MT']), 'body_type'] = 'Sedan'
df.loc[df['full_name'].isin(['2012 Mercedes-Benz SLK-Class SLK 350', '2013 Mercedes-Benz E-Class E350 Cabriolet', 
                             '2009 Mercedes-Benz SLK-Class 55 AMG']), 'body_type'] = 'Convertibles'
df.loc[df['full_name'] == '2011 Jaguar XK R V8 Coupe', 'body_type'] = 'Coupe'
df.loc[df['full_name'].isin(['2015 Chevrolet Enjoy 1.4 LS 7', '2004 Maruti Versa DX2 8-SEATER BSIII TWIN A/C', '2021 Maruti Eeco 5 STR With AC Plus HTR CNG',
                             '2022 Maruti Eeco 5 Seater AC 2020-2022', '2022 Maruti Eeco 5 Seater AC BSVI', '2017 Maruti Eeco Flexi Green', 
                             '2013 Maruti Omni Limited Edition', '2021 Maruti Eeco CNG 5 Seater AC', '2019 Maruti Eeco CNG 5 Seater AC BSIV', 
                             '2019 Maruti Eeco 5 Seater AC', '2013 Maruti Eeco 5 Seater Standard BSIV', '2015 Maruti Omni CNG', 
                             '2015 Maruti Omni MPI CARGO BSIV', '2016 Maruti Eeco CNG 5 Seater AC BSIV', '2013 Toyota Innova 2.5 E Diesel MS 7-seater',
                             '2015 Chevrolet Enjoy 1.4 LS 7']), 'body_type'] = 'Minivans'
df.loc[df['full_name'].isin(['2014 Tata Xenon XT EX 4X2', '2019 Isuzu D-Max High']), 'body_type'] = 'Pickup'

# Check
distinct_full_names_with_blank_body_type = df[df['body_type'].isnull()]['full_name'].unique()
print('Distinct full names with blank body_type:')
for full_name in distinct_full_names_with_blank_body_type:
    print(full_name)

Distinct full names with blank body_type:


In [48]:
# Contar a quantidade de NaN em cada coluna do DataFrame X
nan_count = df.isna().sum()

# Exibir a contagem de NaN por coluna
print(nan_count)

full_name                     0
registered_year              15
engine_capacity               5
insurance                     0
transmission_type             0
kms_driven                    0
owner_type                    9
fuel_type                     0
max_power                    13
seats                         0
mileage                     101
body_type                     0
city                          0
ID                            0
kms_group                     0
id_kms_group                  0
engine_capacity_group         5
id_engine_capacity_group      0
max_power_group              13
id_max_power_group           13
dtype: int64


In [49]:
import pandas as pd

def substituir_valores(df, coluna_alvo, mapeamento):
    df[coluna_alvo] = df[coluna_alvo].replace(mapeamento).fillna(df[coluna_alvo])



# Criar os mapeamentos desejados para cada coluna
mapeamento_id_cidade = dict(zip(dft['city'], dft['id_city']))
mapeamento_id_body_type = dict(zip(dft['body_type'], dft['id_body_type']))
mapeamento_id_owner_type = dict(zip(dft['owner_type'], dft['id_owner_type']))
mapeamento_id_fuel_type = dict(zip(dft['fuel_type'], dft['id_fuel_type']))
mapeamento_id_insurance = dict(zip(dft['insurance'], dft['id_insurance']))



# Chamar a função para substituir os valores nas respectivas colunas
substituir_valores(df, 'city', mapeamento_id_cidade)
substituir_valores(df, 'body_type', mapeamento_id_body_type)
substituir_valores(df, 'owner_type', mapeamento_id_owner_type)
substituir_valores(df, 'fuel_type', mapeamento_id_fuel_type)
substituir_valores(df, 'insurance', mapeamento_id_insurance)


# Visualizar o DataFrame 'df' atualizado
print(df)




                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.445408            Manual     53

In [50]:
import pandas as pd
import re

# Supondo que já carregou os DataFrames df e dft

# Função para extrair a marca a partir da coluna 'full_name' em df
def extrair_marca(nome):
    match = re.search(r'\d{4}\s(.+?)\s', nome)  # Expressão regular para pegar a marca após o ano
    if match:
        return match.group(1)
    else:
        return None

# Aplicar a função para extrair a marca e criar uma nova coluna 'marca' em df
df['marca'] = df['full_name'].apply(extrair_marca)

# Mapear as marcas de df para a coluna 'marca' em dft
mapeamento_marca = dict(zip(dft['marca'], dft['id_categoria_marca']))

# Criar uma nova coluna 'id_categoria_marca' em df com base na correspondência de marcas em dft
df['id_categoria_marca'] = df['marca'].map(mapeamento_marca)

# Visualizar o DataFrame df atualizado
print(df)



                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.445408            Manual     53

In [51]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_categoria_marca'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_categoria_marca'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.445408            Manual     53

In [52]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_id_owner_type_dft = dft['id_owner_type'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_owner_type_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['owner_type'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.445408            Manual     53

In [53]:
#soluçoes id_ENGINE

import pandas as pd

def adicionar_0_1_maior_valorENG(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_id_engine_dft = dft['id_engine_capacity_group'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_engine_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_engine_capacity_group'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valorENG(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.445408            Manual     53

In [54]:
# Converter a coluna 'id_max_power_group' para o tipo de dado float
df['id_max_power_group'] = df['id_max_power_group'].astype(float)


In [55]:
#soluçoes id_maxpower

import pandas as pd

def adicionar_0_1_maior_valormax(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_id_max_power_dft = dft['id_max_power_group'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_max_power_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_max_power_group'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valormax(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.445408            Manual     53

In [56]:
#soluçoes marca

import pandas as pd
import re

# Supondo que já carregou o DataFrame df

# Função para extrair o ano dos primeiros quatro caracteres de 'full_name' em df
def extrair_ano(nome):
    match = re.match(r'\d{4}', nome)  # Expressão regular para pegar os primeiros quatro dígitos (ano)
    if match:
        return match.group()
    else:
        return None

# Aplicar a função para extrair o ano e criar uma nova coluna 'ano' em df
df['ano'] = df['full_name'].apply(extrair_ano)

# Visualizar o DataFrame df com a nova coluna 'ano'
print(df)


                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.445408            Manual     53

In [57]:
# Contar os valores nulos (NaN) em cada coluna
contagem_nan = df.isna().sum()

# Exibir a contagem de valores nulos em cada coluna
print(contagem_nan)

full_name                     0
registered_year              15
engine_capacity               5
insurance                     0
transmission_type             0
kms_driven                    0
owner_type                    0
fuel_type                     0
max_power                    13
seats                         0
mileage                     101
body_type                     0
city                          0
ID                            0
kms_group                     0
id_kms_group                  0
engine_capacity_group         5
id_engine_capacity_group      0
max_power_group              13
id_max_power_group            0
marca                         0
id_categoria_marca            0
ano                           0
dtype: int64


In [58]:
#MILEAGE
#assign value to blank lines
df['mileage'] = df['mileage'].fillna(0)

df = df[df['mileage'].notna()]
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+.\d+|\d+)', str(i))  # Encontrar números (com ou sem ponto decimal)

    if match:
        valor = float(match.group())

        if 'km/kg' in str(i):
            valor *= 1.40  # Converter de km/kg para kmpl

        consumo_correto.append(valor)

df['mileage'] = consumo_correto

In [59]:

#soluçoes id_body_type

import pandas as pd

def adicionar_0_1_maior_valormax(df, dft):
    # Encontrar o maior valor na coluna 'id_body_type' em dft
    maior_valor_id_max_power_dft = dft['id_body_type'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_max_power_dft + 0.1

    # Valores a substituir e seus respectivos substitutos
    substitute_values = {
        'Tata': novo_valor,
        'Mahindra': novo_valor,
        'Maruti': novo_valor,
        'Audi': novo_valor,
        'Toyota': novo_valor,
        'Honda': novo_valor,
        'BMW': novo_valor,
        'Datsun': novo_valor,
        'Chevrolet': novo_valor,
        'Jaguar': novo_valor,
        'Porsche': novo_valor
    }

    # Substituir os valores na coluna 'body_type' pelos valores do dicionário substitute_values
    df['body_type'].replace(substitute_values, inplace=True)



# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valormax(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.445408            Manual     53

In [60]:
# Obtendo os valores únicos da coluna 'body_type'
valores_unicos_body_type = df['body_type'].unique()

# Exibindo os valores únicos
print(valores_unicos_body_type)


[0.36411207 0.59886912 3.75079304 0.57805311 1.34759698 2.2890652
 3.28281625 3.04873305 3.42894429]


In [61]:
df

,full_name,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,...,ID,kms_group,id_kms_group,engine_capacity_group,id_engine_capacity_group,max_power_group,id_max_power_group,marca,id_categoria_marca,ano
0,2017 Hyundai Grand i10 1.2 Kappa Sportz Option,Jun 2017,1197.0,0.445408,Manual,53658.0,0.150465,0.180334,81.8,5.0,...,0,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055,Hyundai,0.073531,2017
1,2020 Maruti Swift AMT VXI,Feb 2021,1197.0,1.418644,Automatic,33055.0,0.620328,0.180334,81.8,5.0,...,1,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055,Maruti,0.073531,2020
2,2010 Maruti Zen Estilo LX BSIII,Jul 2010,998.0,0.256633,Manual,90385.0,1.365236,0.180334,67.1,5.0,...,2,Até 150.000 km,0.005469,até 1000 cc,0.776698,0-200 bhp,0.009055,Maruti,0.073531,2010
3,2010 Maruti Alto LXi,2010,796.0,0.256633,Manual,110000.0,0.150465,0.180334,46.3,5.0,...,3,Até 150.000 km,0.005469,até 1000 cc,0.776698,0-200 bhp,0.009055,Maruti,0.073531,2010
4,2018 Hyundai i20 Active S Petrol,Oct 2018,1197.0,0.445408,Manual,58000.0,0.620328,0.180334,81.8,5.0,...,4,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055,Hyundai,0.073531,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,2022 Tata Tigor EV XZ Plus,2022,NaN,0.256633,Automatic,30000.0,0.150465,2.455816,73.7,5.0,...,3485,Até 150.000 km,0.005469,<NA>,4.127914,0-200 bhp,0.009055,Tata,0.073531,2022
3486,2017 Renault KWID 1.0 RXT,Apr 2017,999.0,0.256633,Manual,64024.0,0.150465,0.180334,67.0,5.0,...,3486,Até 150.000 km,0.005469,até 1000 cc,0.776698,0-200 bhp,0.009055,Renault,0.073531,2017
3487,2017 Honda City i VTEC VX,Dec 2017,1497.0,0.445408,Manual,39000.0,0.150465,0.180334,117.3,5.0,...,3487,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055,Honda,0.073531,2017
3488,2016 Honda Jazz 1.2 V i VTEC,Jun 2016,1199.0,0.445408,Manual,33000.0,0.150465,0.180334,88.7,5.0,...,3488,Até 150.000 km,0.005469,até 3000 cc,0.082826,0-200 bhp,0.009055,Honda,0.073531,2016


In [62]:
# Contar a quantidade de NaN em cada coluna do DataFrame X
nan_count = df.isna().sum()

# Exibir a contagem de NaN por coluna
print(nan_count)

full_name                    0
registered_year             15
engine_capacity              5
insurance                    0
transmission_type            0
kms_driven                   0
owner_type                   0
fuel_type                    0
max_power                   13
seats                        0
mileage                      0
body_type                    0
city                         0
ID                           0
kms_group                    0
id_kms_group                 0
engine_capacity_group        5
id_engine_capacity_group     0
max_power_group             13
id_max_power_group           0
marca                        0
id_categoria_marca           0
ano                          0
dtype: int64


In [63]:
df = df.astype({
    
    'ano': float,
    'body_type': float,
    
  
})

In [64]:
#Export
#Not the target
colunas_selecionadas = ['body_type','id_categoria_marca','ano','city','insurance','seats','mileage','id_engine_capacity_group','fuel_type','id_kms_group','id_max_power_group','owner_type','ID']

if set(colunas_selecionadas).issubset(df.columns):
    df_selecionado = df[colunas_selecionadas]
    df_selecionado.to_csv('testcenario2MenosGrupos.csv', index=False)

In [65]:
df_colunas_selecionadas = pd.DataFrame(colunas_selecionadas)

# Contar a quantidade de NaN em cada coluna do DataFrame df_colunas_selecionadas
nan_count = df_colunas_selecionadas.isna().sum()

# Exibir a contagem de NaN por coluna
print(nan_count)


0    0
dtype: int64
